<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/cDNA-pretraining/blob/main/models/training_20220714/Training_with_BPE__10k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq transformers datasets comet_ml

     |████████████████████████████████| 4.4 MB 14.0 MB/s 
     |████████████████████████████████| 362 kB 64.6 MB/s 
     |████████████████████████████████| 372 kB 73.5 MB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 
     |████████████████████████████████| 6.6 MB 58.8 MB/s 
     |████████████████████████████████| 212 kB 27.1 MB/s 
     |████████████████████████████████| 140 kB 45.1 MB/s 
     |████████████████████████████████| 1.1 MB 60.9 MB/s 
     |████████████████████████████████| 127 kB 78.3 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 146 kB 66.0 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 554 kB 56.4 MB/s 
     |████████████████████████████████| 144 kB 68.4 MB/s 
     |████████████████████████████████| 271 kB 80.4 MB/s 
     |████████████████████████████████| 94 kB 4.7 MB/s 
ERROR: pip's depende

In [1]:
# PARAMETERS

DATASET = "simecek/Human_DNA_v0_BPEtokenized_vocab10k"
HF_MODEL_NAME = "DNADebertaBPE10k"

LR = 5e-05
WD = 0.1

BATCH_SIZE = 16
ACCUMULATION = 4
HIDDEN_LAYERS = 6
MLM_P = 0.15
EPOCHS = 15

## 0) Loading

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
import os

os.environ['COMET_API_KEY'] = "uckBYzSuTQ0uUKVA42536dD7Q"

import comet_ml

# Commet Init
comet_ml.init(project_name="Training_20220714", api_key="uckBYzSuTQ0uUKVA42536dD7Q")

COMET INFO: Comet API key is valid
COMET WARNING: running in Google Colab, but can't find mounted drive. Using HOME instead
COMET WARNING: if drive is mounted, set COMET_CONFIG to save config there
COMET INFO: Comet API key saved in /root/.comet.config


In [4]:
from datasets import load_dataset
datasets = load_dataset(DATASET)
datasets.set_format("torch")
datasets

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Using custom data configuration simecek--Human_DNA_v0_BPEtokenized_vocab10k-4be48be7026338c9


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/simecek___parquet/simecek--Human_DNA_v0_BPEtokenized_vocab10k-4be48be7026338c9/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1005966
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 111765
    })
})

In [5]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("Vlasta/DNA_BPE_max_vocab_10000_maxk_10_max_tokenlen_10")

Downloading:   0%|          | 0.00/257 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

## 1) Training

In [6]:
from transformers import DebertaConfig, DebertaForMaskedLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

model_config = DebertaConfig(vocab_size=len(tokenizer.vocab), max_position_embeddings=512, num_hidden_layers=HIDDEN_LAYERS)
model_config

DebertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": null,
  "position_biased_input": true,
  "relative_attention": false,
  "transformers_version": "4.20.1",
  "type_vocab_size": 0,
  "vocab_size": 10000
}

In [7]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=MLM_P)

model = DebertaForMaskedLM(config=model_config)
sum(p.numel() for p in model.parameters()) // 10**6

51

In [8]:
tokenizer.set_truncation_and_padding

<bound method PreTrainedTokenizerFast.set_truncation_and_padding of PreTrainedTokenizerFast(name_or_path='Vlasta/DNA_BPE_max_vocab_10000_maxk_10_max_tokenlen_10', vocab_size=10000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})>

In [9]:
training_args = TrainingArguments(
    output_dir='./model',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=EPOCHS,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=BATCH_SIZE, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=ACCUMULATION,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=BATCH_SIZE,  # evaluation batch size
    logging_steps=5000,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=5000,
    fp16=True,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=1,           # whether you don't have much space so you let only 5 model weights saved in the disk
    push_to_hub=True,
    hub_model_id=HF_MODEL_NAME,
    hub_strategy="every_save",
    learning_rate=LR,
    weight_decay=WD
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=datasets['train'],
    eval_dataset=datasets['test'],
)

Cloning https://huggingface.co/simecek/DNADebertaBPE10k into local empty directory.
Using cuda_amp half precision backend


In [11]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1005966
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 235770
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration 

Step,Training Loss,Validation Loss
5000,7.280800,7.205214
10000,7.191200,7.187807
15000,7.175700,7.178442
20000,7.166300,7.171922
25000,7.159500,7.162162
30000,7.070200,6.931949
35000,6.477600,5.739536
40000,5.615700,5.399255
45000,5.351600,5.211689
50000,5.202400,5.107907


***** Running Evaluation *****
  Num examples = 111765
  Batch size = 16
Saving model checkpoint to ./model/checkpoint-5000
Configuration saved in ./model/checkpoint-5000/config.json
Model weights saved in ./model/checkpoint-5000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 111765
  Batch size = 16
Saving model checkpoint to ./model/checkpoint-10000
Configuration saved in ./model/checkpoint-10000/config.json
Model weights saved in ./model/checkpoint-10000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 111765
  Batch size = 16
Saving model checkpoint to ./model/checkpoint-15000
Configuration saved in ./model/checkpoint-15000/config.json
Model weights saved in ./model/checkpoint-15000/pytorch_model.bin
Deleting older checkpoint [model/checkpoint-5000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 111765
  Batch size = 16
Saving model checkpoint to ./model/checkpoint-20000
Configuration saved in ./model/checkpoint-2000

Step,Training Loss,Validation Loss
5000,7.280800,7.205214
10000,7.191200,7.187807
15000,7.175700,7.178442
20000,7.166300,7.171922
25000,7.159500,7.162162
30000,7.070200,6.931949
35000,6.477600,5.739536
40000,5.615700,5.399255
45000,5.351600,5.211689
50000,5.202400,5.107907


***** Running Evaluation *****
  Num examples = 111765
  Batch size = 16
Saving model checkpoint to ./model/checkpoint-110000
Configuration saved in ./model/checkpoint-110000/config.json
Model weights saved in ./model/checkpoint-110000/pytorch_model.bin
Deleting older checkpoint [model/checkpoint-100000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 111765
  Batch size = 16
Saving model checkpoint to ./model/checkpoint-115000
Configuration saved in ./model/checkpoint-115000/config.json
Model weights saved in ./model/checkpoint-115000/pytorch_model.bin
Deleting older checkpoint [model/checkpoint-105000] due to args.save_total_limit


KeyboardInterrupt: ignored

In [12]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 111765
  Batch size = 16


Step,Training Loss,Validation Loss
5000,7.280800,7.205214
10000,7.191200,7.187807
15000,7.175700,7.178442
20000,7.166300,7.171922
25000,7.159500,7.162162
30000,7.070200,6.931949
35000,6.477600,5.739536
40000,5.615700,5.399255
45000,5.351600,5.211689
50000,5.202400,5.107907


{'eval_loss': 4.732296466827393}

In [13]:
trainer.push_to_hub()

Saving model checkpoint to ./model
Configuration saved in ./model/config.json
Model weights saved in ./model/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/195M [00:00<?, ?B/s]

Upload file runs/Jul14_08-45-47_903978113fc8/events.out.tfevents.1657788364.903978113fc8.188.0:  25%|##4      …

To https://huggingface.co/simecek/DNADebertaBPE10k
   2f8dc32..3ec7c06  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}}
To https://huggingface.co/simecek/DNADebertaBPE10k
   3ec7c06..b8fec7d  main -> main



'https://huggingface.co/simecek/DNADebertaBPE10k/commit/3ec7c0656748600046c6aeaccf8e7846a101e8ef'